In [13]:
import pandas as pd
import re
import string
import matplotlib.pyplot as plt
import numpy as np
import spacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
df = pd.read_csv('data/cannabis.csv')

In [3]:
df = df.replace('None', np.nan)
df = df.dropna().reset_index()

In [4]:
df['Flavor']= df['Flavor'].str.replace(',',' ')
df['Effects'] = df['Effects'].str.replace(',', ' ')

In [5]:
df['combined'] = df["Effects"] +  df["Flavor"] + df['Description']

In [15]:
df.head()

,index,Strain,Type,Rating,Effects,Flavor,Description,combined
0,0,100-Og,hybrid,4.0,Creative Energetic Tingly Euphoric Relaxed,Earthy Sweet Citrus,$100 OG is a 50/50 hybrid strain that packs a ...,Creative Energetic Tingly Euphoric RelaxedEart...
1,1,98-White-Widow,hybrid,4.7,Relaxed Aroused Creative Happy Energetic,Flowery Violet Diesel,The ‘98 Aloha White Widow is an especially pot...,Relaxed Aroused Creative Happy EnergeticFlower...
2,2,1024,sativa,4.4,Uplifted Happy Relaxed Energetic Creative,Spicy/Herbal Sage Woody,1024 is a sativa-dominant hybrid bred in Spain...,Uplifted Happy Relaxed Energetic CreativeSpicy...
3,3,13-Dawgs,hybrid,4.2,Tingly Creative Hungry Relaxed Uplifted,Apricot Citrus Grapefruit,13 Dawgs is a hybrid of G13 and Chemdawg genet...,Tingly Creative Hungry Relaxed UpliftedApricot...
4,4,24K-Gold,hybrid,4.6,Happy Relaxed Euphoric Uplifted Talkative,Citrus Earthy Orange,"Also known as Kosher Tangie, 24k Gold is a 60%...",Happy Relaxed Euphoric Uplifted TalkativeCitru...


In [6]:
tfidf = TfidfVectorizer(stop_words = 'english',ngram_range = (1,2),max_features = 2000)
dtm = tfidf.fit_transform(df['combined'])
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
nn = NearestNeighbors(n_neighbors=4, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)

In [7]:
import pickle
# Dump the trained classifier (nn)  with Pickle
pickle_filename = 'model.pkl2'
pickled_model = open(pickle_filename, 'wb')  # Open the file to save as pkl file
pickle.dump(nn, pickled_model)
pickled_model.close() # Close the pickle instances

In [8]:
# Loading the saved model
model_pkl2 = open(pickle_filename, 'rb')
NN_model2 = pickle.load(model_pkl2)
print ("Loaded model :: ", NN_model2)  # print to verify

Loaded model ::  NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=4, p=2,
                 radius=1.0)


In [9]:
# Dump the trained classifier (tfidf)  with Pickle
pickle_filename_1 = 'tfidf.pkl2'
pickled_model_1 = open(pickle_filename_1, 'wb')  # Open the file to save as pkl file
pickle.dump(tfidf, pickled_model_1)
pickled_model_1.close() # Close the pickle instances

In [10]:
# Loading the saved model
model_pkl_1 = open(pickle_filename_1, 'rb')
tfidf_model2 = pickle.load(model_pkl_1)
print ("Loaded model :: ", tfidf_model2)  # print to verify

Loaded model ::  TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)


In [16]:
import json
def recommend(user_input):
    temp_df = NN_model2.kneighbors(tfidf_model2.transform([user_input]).todense())[1]
    

    #print(temp_df)
    
    for i in range(4):
        info = df.iloc[temp_df[0][i]]['Strain']
        info_effects = df.iloc[temp_df[0][i]]['Effects']
        info_flavor = df.iloc[temp_df[0][i]]['Flavor']
        info_description = df.iloc[temp_df[0][i]]['Description']
        info_rating = df.iloc[temp_df[0][i]]['Rating']
        
        print(json.dumps(info))
        print(json.dumps(info_effects))
        print(json.dumps(info_flavor))
        print(json.dumps(info_description))
        print(json.dumps(info_rating))
        
        #return json.dumps(info)  #for engineeers, the return does not work in jupyter lab.  Should work in vsCode.
        #return json.dumps(info_effects)
        #return json.dumps(info_flavor)
        #return json.dump(info_description)
        #return json.dumps(info_rating)

In [17]:
recommend('relieving stress and muscle tension')

"Alpine-Star"
"Relaxed Uplifted Happy Euphoric Tingly"
"Pine Citrus Earthy"
"Alpine Star, often labeled as Alpine OG, is an earthy, citrus mix of Tahoe OG and Sensi Star genetics. Zesty lemon flavors lead Alpine Star\u2019s wave of relaxing indica effects. The pain relieving properties of Alpine Star radiate from the head and neck, calming anxieties and muscle tension as the effects descend to the lower extremities."
4.2
"Saturn-Og"
"Happy Relaxed Euphoric Uplifted Sleepy"
"Earthy Sweet Tree Fruit"
"Saturn OG is a hybrid strain with mysterious beginnings, but its earthy citrus and diesel flavors confirms its close relationship to OG Kush. Its undocumented origins result in this strain being labeled as indica, sativa, and everything in between, but its effects are most commonly described as hybrid-like in its balanced calm and moderate cerebral effects. Saturn OG\u2019s forest green buds are lit by a constellation of crystal trichomes, and this OG Kush relative is often lumped into a \u